In [ ]:
from PIL import Image
from glob import glob
import os
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from torch import nn
from torchvision import models
from torchsummary import summary
from torchvision import transforms
from torchvision.transforms import ToPILImage
from torch.utils.data import Dataset, DataLoader
import os
import cv2
import matplotlib.pyplot as plt
import math
import random
import numpy as np
import sys
import json
import time

In [ ]:
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
TORCH_CUDA_ARCH_LIST="8.6"

tl=time.localtime()
format_time = time.strftime("%Y%m%d_%H%M%S", tl)

In [ ]:
# Function to convert words to their corresponding indices and vice versa
def word_to_ids(word_counts, unk_required):
    # Initialize dictionaries to store word-to-index and index-to-word mappings
    word_to_id = {}
    id_to_word = {}
    count = 0  
    
    # Add <UNK> token if required
    if unk_required:
        word_to_id['<UNK>'] = count  
        id_to_word[count] = '<UNK>' 
        count += 1  
        print("<UNK> True")  
    
    # Iterate through each word in the word_counts list
    for word in word_counts:
        word_to_id[word] = count 
        id_to_word[count] = word 
        count += 1  
    
    # Return dictionaries containing word-to-index and index-to-word mappings
    return word_to_id, id_to_word


In [ ]:
# Function to build vocabulary based on the training and test sets
def build_vocab(training_set, test_set, word_count_threshold, unk_required=False):
    dataset_all = training_set + test_set
    
    # Extract all sentences from the dataset
    sentences_all = []
    for i in range(len(dataset_all)):
        sub_sentence = dataset_all[i]['caption']
        for j in sub_sentence:
            sentences_all.append(j)
    
    # Process captions to build word counts
    all_captions = []
    word_counts = {}
    for caption in sentences_all:
        caption = '<BOS> ' + caption.split('\t')[-1] + ' <EOS>'
        all_captions.append(caption)
        
        # Update word counts
        for word in caption.split(' '):
            if word in word_counts:
                word_counts[word] += 1
            else:
                word_counts[word] = 1
    
    # Remove words with counts below the threshold
    for word in list(word_counts.keys()):
        if word_counts[word] < word_count_threshold:
            word_counts.pop(word)
            unk_required = True  
    
    # Return word counts and unk_required flag
    return word_counts, unk_required

In [ ]:
# Function to fetch features of training videos along with their captions
def get_train_data(batch_size):
    vid = np.random.choice(video_train, batch_size)  
    cur_vid = np.array([np.load(VIDEO_DIR_TRAIN + video + '.npy') for video in vid])  # Load video features
    feats_idx = np.linspace(0, 79, n_lstm_steps).astype(int)  
    cur_vid = cur_vid[:, feats_idx, :]  
    captions = [np.random.choice(Vid2Cap_train[video], 1)[0] for video in vid]  
    captions_list = captions  
    captions, cap_mask = convert_caption(captions, word2id, n_lstm_steps)  
    return cur_vid, captions, cap_mask, captions_list, vid  # Return video features, caption sequences, mask, caption list, and video IDs

In [ ]:
# Function to fetch features of test videos along with their captions
def get_test_data(batch_size):
    vid = np.random.choice(video_test, batch_size)  # Select random test videos
    cur_vid = np.array([np.load(VIDEO_DIR_TEST + video + '.npy') for video in vid]) 
    feats_idx = np.linspace(0, 79, n_lstm_steps).astype(int) 
    cur_vid = cur_vid[:, feats_idx, :]  
    captions = [np.random.choice(Vid2Cap_test[video], 1)[0] for video in vid] 
    captions_list = captions 
    captions, cap_mask = convert_caption(captions, word2id, n_lstm_steps)  
    return cur_vid, captions, cap_mask, captions_list, vid  # Return video features, caption sequences

In [ ]:
# Function to convert captions to sequences of word indices and generate corresponding masks
def convert_caption(captions, word_to_id, max_length):
    if type(captions) == 'str':
        captions = [captions]
    caps, cap_mask = [], []
    for cap in captions:
        nWord = len(cap.split(' ')) 
        cap = cap + ' <EOS>'*(max_length-nWord) 
        cap_mask.append([1.0]*nWord + [0.0]*(max_length-nWord))  
        cap_ids = []
        for word in cap.split(' '):  
            if word in word_to_id:  
                cap_ids.append(word_to_id[word])  
            else:
                cap_ids.append(word_to_id['<UNK>'])  
        caps.append(cap_ids)  
    return np.array(caps), np.array(cap_mask)  # Return arrays of word indices and mask

In [ ]:
# print captions
def print_in_english(captions):
    captions_english = [[id2word[word] for word in caption] for caption in captions]
    cap_list = []
    for cap in captions_english:
        if '<EOS>' in cap:
            cap = cap[0:cap.index('<EOS>')]
        cap_list.append((' ' + ' '.join(cap)))
    return cap_list

In [ ]:
# Function to extract a frame from a video
def get_frame(video_name):
    testvideo = cv2.VideoCapture(video_name)  # Open the video file
    if testvideo.isOpened():
        frames = int(testvideo.get(cv2.CAP_PROP_FRAME_COUNT))  
        width = testvideo.get(cv2.CAP_PROP_FRAME_WIDTH)  
        height = testvideo.get(cv2.CAP_PROP_FRAME_HEIGHT)  
        frame = 0  
        while 1:
            if frame == 15:  
                ret, frame_img = testvideo.read() 
                break
            else:
                frame = frame + 1  
    return frame_img  # Return the captured frame

In [ ]:
# save training log
def write_txt(epoch, iteration, loss, time_cost):
    with open("./save_text/train_log_%s.txt"%(format_time), 'a') as f:
        f.write("Epoch:[ %d ]\t Iteration:[ %d ]\t loss:[ %f ]\t time:[ %f ]\n" % (epoch, iteration, loss, time_cost))
        
def output_txt(filename, video_name, captions):
    with open("./%s"%(filename), 'a') as f:
        f.write("%s,%s\n" % (video_name, captions[0]))

In [ ]:

def batch_test_vis(vid):
    frames_1 = np.array([])
    for i in range(0,5):
        frame_i = get_frame(VIDEO_TEST+vid[i])
        frame_i = cv2.resize(frame_i, (128,128))
        if i == 0:
            frames_1 = frame_i
        else:
            frames_1 = np.vstack((frames_1,frame_i))
            
    frames_2 = np.array([])            
    for i in range(5,10):
        frame_i = get_frame(VIDEO_TEST+vid[i])
        frame_i = cv2.resize(frame_i, (128,128))
        if i == 5:
            frames_2 = frame_i
        else:
            frames_2 = np.vstack((frames_2,frame_i))
        
    frames = np.hstack((frames_1,frames_2))
    cell = Image.fromarray(cv2.cvtColor(frames, cv2.COLOR_BGR2RGB))
    return frames, cell

In [ ]:
def batch_train_vis(vid):
    frames_1 = np.array([])
    for i in range(0,5):
        frame_i = get_frame(VIDEO_TRAIN+vid[i])
        frame_i = cv2.resize(frame_i, (128,128))
        if i == 0:
            frames_1 = frame_i
        else:
            frames_1 = np.vstack((frames_1,frame_i))
            
    frames_2 = np.array([])            
    for i in range(5,10):
        frame_i = get_frame(VIDEO_TRAIN+vid[i])
        frame_i = cv2.resize(frame_i, (128,128))
        if i == 5:
            frames_2 = frame_i
        else:
            frames_2 = np.vstack((frames_2,frame_i))
        
    frames = np.hstack((frames_1,frames_2))
    cell = Image.fromarray(cv2.cvtColor(frames, cv2.COLOR_BGR2RGB))
    return frames, cell

In [ ]:
class S2VT(nn.Module):
    def __init__(self, vocab_size, batch_size=10, frame_dim=4096, hidden=500, dropout=0.5, n_step=80):
        super(S2VT, self).__init__()
        self.batch_size = batch_size
        self.frame_dim = frame_dim
        self.hidden = hidden
        self.n_step = n_step

        self.drop = nn.Dropout(p=dropout)
        self.linear1 = nn.Linear(frame_dim, hidden)
        self.linear2 = nn.Linear(hidden, vocab_size)

        self.lstm1 = nn.LSTM(hidden, hidden, batch_first=True, dropout=dropout)
        self.lstm2 = nn.LSTM(2*hidden, hidden, batch_first=True, dropout=dropout)

        self.embedding = nn.Embedding(vocab_size, hidden)

    def forward(self, video, caption=None):
        video = video.contiguous().view(-1, self.frame_dim)
        video = self.drop(video)
        video = self.linear1(video)                   
        video = video.view(-1, self.n_step, self.hidden)
        padding = torch.zeros([self.batch_size, self.n_step-1, self.hidden]).cuda()
        video = torch.cat((video, padding), 1)        
        vid_out, state_vid = self.lstm1(video)

        if self.training:
            caption = self.embedding(caption[:, 0:self.n_step-1])
            padding = torch.zeros([self.batch_size, self.n_step, self.hidden]).cuda()
            caption = torch.cat((padding, caption), 1)      
            caption = torch.cat((caption, vid_out), 2)        

            cap_out, state_cap = self.lstm2(caption)
            cap_out = cap_out[:, self.n_step:, :]
            cap_out = cap_out.contiguous().view(-1, self.hidden)
            cap_out = self.drop(cap_out)
            cap_out = self.linear2(cap_out)
            return cap_out
        else:
            padding = torch.zeros([self.batch_size, self.n_step, self.hidden]).cuda()
            cap_input = torch.cat((padding, vid_out[:, 0:self.n_step, :]), 2)
            cap_out, state_cap = self.lstm2(cap_input)

            bos_id = word2id['<BOS>']*torch.ones(self.batch_size, dtype=torch.long)
            bos_id = bos_id.cuda()
            cap_input = self.embedding(bos_id)
            cap_input = torch.cat((cap_input, vid_out[:, self.n_step, :]), 1)
            cap_input = cap_input.view(self.batch_size, 1, 2*self.hidden)

            cap_out, state_cap = self.lstm2(cap_input, state_cap)
            cap_out = cap_out.contiguous().view(-1, self.hidden)
            cap_out = self.drop(cap_out)
            cap_out = self.linear2(cap_out)
            cap_out = torch.argmax(cap_out, 1)

            caption = []
            caption.append(cap_out)
            for i in range(self.n_step-2):
                cap_input = self.embedding(cap_out)
                cap_input = torch.cat((cap_input, vid_out[:, self.n_step+1+i, :]), 1)
                cap_input = cap_input.view(self.batch_size, 1, 2 * self.hidden)

                cap_out, state_cap = self.lstm2(cap_input, state_cap)
                cap_out = cap_out.contiguous().view(-1, self.hidden)
                cap_out = self.drop(cap_out)
                cap_out = self.linear2(cap_out)
                cap_out = torch.argmax(cap_out, 1)
                caption.append(cap_out)
            return caption  

In [ ]:
if __name__ == "__main__":
    time_start_all = time.time()
    time_start = time.time()

    # Handling command line arguments for data_path and output_filename
    if len(sys.argv) >= 2:
        data_path = './' + str(sys.argv[1]) + '/'
    else:
        data_path = './MLDS_hw2_1_data/'

    if len(sys.argv) >= 3:
        output_filename = str(sys.argv[2])
    else:
        output_filename = 'test_output.txt'
    
    print('load successful...')
    print('data_path = ', data_path)
    print('output_filename = ', output_filename)
        
    n_lstm_steps = 80
    model_path = './'
    model_file = 's2vt_best.pth'
    VIDEO_TRAIN = data_path + 'training_data/video/'
    VIDEO_TEST  = data_path + 'testing_data/video/'
    VIDEO_DIR_TRAIN = data_path + 'training_data/feat/'
    VIDEO_DIR_TEST  = data_path + 'testing_data/feat/'
    training_set = json.load(open(data_path + 'training_label.json', 'r'))
    test_Set  = json.load(open(data_path + 'testing_label.json', 'r'))
    video_train = list(open(data_path + 'training_data/id.txt', 'r').read().splitlines())
    video_test  = list(open(data_path + 'testing_data/id.txt', 'r').read().splitlines())
    
    # Build vocabulary
    Vid2Cap_train = {}
    for i in range(len(training_set)):
        vid = training_set[i]['id']
        cap = training_set[i]['caption']
        for j in range(len(cap)):
            cap[j] = '<BOS> ' + cap[j] + ' <EOS>'
        Vid2Cap_train[vid] = cap
        
    Vid2Cap_test = {}
    for i in range(len(test_Set)):
        vid = test_Set[i]['id']
        cap = test_Set[i]['caption']
        for j in range(len(cap)):
            cap[j] = '<BOS> ' + cap[j] + ' <EOS>'
        Vid2Cap_test[vid] = cap
        
    word_counts, unk_required = build_vocab(training_set, test_Set, word_count_threshold=0)
    word2id, id2word = word_to_ids(word_counts, unk_requried=unk_required)

    # Hyperparameters
    EPOCH = 30
    nIter = 2000
    BATCH_SIZE = 10
    LEARNING_RATE = 0.0001
    
    vocab_size = len(word_counts)
    import_model = True
    show_image_wall = False

    # Initialize model
    s2vt = S2VT(vocab_size=vocab_size, batch_size=BATCH_SIZE)
    if import_model:
        s2vt.load_state_dict(torch.load(model_path + model_file))
    s2vt = s2vt.cuda()
    loss_func = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(s2vt.parameters(), lr=LEARNING_RATE)
    loss_list = []

    # Training loop
    for epoch in range(EPOCH):
        for i in range(nIter):
            video, caption, cap_mask, captions_list, vid = get_train_data(BATCH_SIZE)
            video, caption, cap_mask = torch.FloatTensor(video).cuda(), torch.LongTensor(caption).cuda(), torch.FloatTensor(cap_mask).cuda()
            cap_out = s2vt(video, caption)
            cap_labels = caption[:, 1:].contiguous().view(-1)      
            cap_mask = cap_mask[:, 1:].contiguous().view(-1)       
            logit_loss = loss_func(cap_out, cap_labels)
            masked_loss = logit_loss * cap_mask
            loss = torch.sum(masked_loss) / torch.sum(cap_mask)
            loss_list.append(loss.detach().cpu().numpy())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if i % nIter == 0:
                time_end = time.time()              
                time_cost = time_end - time_start
                print("Epoch: %d  iteration: %d , loss: %f, time: %f" % (epoch, i, loss, time_cost))
                time_start = time.time()
                write_txt(epoch, i, loss, time_cost)
                
            if i % nIter == 0:
                torch.save(s2vt.state_dict(), model_path + model_file)
                print("Epoch: %d save successed!" % (epoch))

    time_end_all = time.time()
    print("Training Done! Total time cost: %f" % (time_end_all - time_start_all))
    
    # Save test output to text
    BATCH_SIZE = 1

    s2vt = S2VT(vocab_size=vocab_size, batch_size=BATCH_SIZE)
    s2vt = s2vt.cuda()
    s2vt.load_state_dict(torch.load(model_path + model_file))
    s2vt.eval()

    for i in range(len(video_test)):
        video_name = video_test[i]
        video = np.array(np.load(VIDEO_DIR_TEST + video_name + '.npy'))
        video = torch.FloatTensor(video).cuda()
        cap_out = s2vt(video)
        captions = []
        for tensor in cap_out:
            captions.append(tensor.tolist())
        captions = [[row[i] for row in captions] for i in range(len(captions[0]))]
        output = print_in_english(captions)
  
        output_txt(output_filename, video_name, output)
    print("Test Done! Output text file established!")
